# HRV Investigator - Overview


# Imports / Installs

In [ ]:
import sys
import importlib.util
import numpy as np
import panel as pn
import param
import ipywidgets as ipywidget
import pickle
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource
import os

# def checkAndInstallPip(package_name):
#     if importlib.util.find_spec(package_name) is not None:
#         print(f"{package_name!r} already installed")
#     else:        
#         print(f"{package_name!r} will be installed")
#         !{sys.executable} - m pip install {package_name}


# def checkAndInstallConda(package_name, channel=None):
#     if importlib.util.find_spec(package_name) is not None:
#         print(f"{package_name!r} already installed")
#     else:
#         print(f"{package_name!r} will be installed")
#         if channel is not None:
#             %conda install - -yes - -prefix {sys.prefix} - c {channel} {package_name}
#         else:
#             %conda install - -yes - -prefix {sys.prefix} {package_name}

# try:
#     checkAndInstallConda("ipywidgets", sys.prefix)
# except ImportError as e:
#     print("error", e)
#     pass

# try:
#     checkAndInstallPip("numpy")
#     checkAndInstallPip("mne")
#     checkAndInstallPip("bqplot")
#     checkAndInstallPip("holoviews")
# except ImportError as e:
#     print("error", e)
#     pass

In [ ]:
# Panel Initialisation
#pn.extension( css_files=["./style/dist/hrv.css"])

pn.extension('terminal', 'gridstack')
pn.config.comms = "vscode"
pn.param.ParamMethod.loading_indicator = True


# Data Layout

In [ ]:
import param
from components.dataStructure import DataHistoryItem, DataHistoryList
from components.common.activeECGDataSelector import ECGHistorySelector, ECGRunSelector

# Layout panes
# Subcompoents of these panels are defined in their respective section
processing_panel = None
insights_panel = None

ml_panel = None
data_panel = None  # For managing storing and loading data
console_panel = None  # For displaying messages and errors
dashboard = None

# Main DataFrame
main_ecg_history = DataHistoryList()  

# pathDefinition
absPath = os.path.abspath('')

# Temp - DataLoading

In [ ]:
import mne
import random
import matplotlib.pyplot as plt
import pandas as pd
import plotly.express as px
import plotly.subplots as sp

def loadData():
    import h5py
    import numpy as np
    import os
    absPath=""
    
    fileName = "/Users/shivam/Documents/GitHub/HRVInvestigator/DataArtifacts/demo_data.pickle"
    # combine to total path 
    filePath = os.path.join(absPath, fileName)
    
    file = h5py.File('/Users/shivam/Documents/GitHub/HRVInvestigator/DataArtifacts/ecg.mat', 'r')

    # Access the ECG data
    ecg_data = file['ecg'][:].flatten()
    
    
    data_obj = main_ecg_history.addNewFromDict({"data": ecg_data, "features": {"sfreq": file["header"]["sampling_rate"][0]}, "name": "ECG", "mlRuns": {}})
    file.close()
    # print("HERE")
    # with open(filePath, "wb") as f:
    #         pickle.dump(data_obj.getSaveDict(), f, pickle.HIGHEST_PROTOCOL)
    
def loadfeature30data():
    filepath = "/Users/shivam/Documents/GitHub/HRVInvestigator/DataArtifacts/features_30.pkl"
    obj = pd.read_pickle(filepath)
    # obj.plot(subplots=True, kind = "kde")
    fig = sp.make_subplots(rows=int(obj.shape[1]/3), cols=3)
    i=0
    j=0
    for col in obj.columns:
        hist1 = px.histogram(obj[col], x=col, histnorm='density', nbins=1000)
        fig.add_trace(hist1.data[0], row=i+1, col=j+1)
        fig.update_xaxes(title_text=col, row=i+1, col=j+1)
        # subobj.plot(kind="kde" ,ax=axes[i,j],legend=True)
        j+=1
        if j == 3:
            i+=1
            j=0
    fig.update_layout(title='30s Features', height=600)
    return fig

def loadfeature60data():
    filepath = "/Users/shivam/Documents/GitHub/HRVInvestigator/DataArtifacts/features_60.pkl"
    obj = pd.read_pickle(filepath)
    # obj.plot(subplots=True, kind = "kde")
    fig = sp.make_subplots(rows=int(obj.shape[1]/3), cols=3)
    i=0
    j=0
    for col in obj.columns:
        hist1 = px.histogram(obj[col], x=col, histnorm='density', nbins=1000)
        fig.add_trace(hist1.data[0], row=i+1, col=j+1)
        fig.update_xaxes(title_text=col, row=i+1, col=j+1)
        # subobj.plot(kind="kde" ,ax=axes[i,j],legend=True)
        j+=1
        if j == 3:
            i+=1
            j=0
    fig.update_layout(title='60s Features', height=600)
    return fig

def loadfeature120data():
    filepath = "/Users/shivam/Documents/GitHub/HRVInvestigator/DataArtifacts/features_120.pkl"
    obj = pd.read_pickle(filepath)
    # obj.plot(subplots=True, kind = "kde")
    fig = sp.make_subplots(rows=int(obj.shape[1]/3), cols=3)
    i=0
    j=0
    for col in obj.columns:
        hist1 = px.histogram(obj[col], x=col, histnorm='density', nbins=1000)
        fig.add_trace(hist1.data[0], row=i+1, col=j+1)
        fig.update_xaxes(title_text=col, row=i+1, col=j+1)
        # subobj.plot(kind="kde" ,ax=axes[i,j],legend=True)
        j+=1
        if j == 3:
            i+=1
            j=0
    fig.update_layout(title='120s Features', height=600)
    return fig

loadData()
# loadfeature30data()
main_ecg_history.current.data = main_ecg_history.current.data[0:80000:1]
# main_ecg_history.current.mlRuns["truth"] = main_ecg_history.current.mlRuns["truth"][10000:20000:1]
# main_ecg_history.current.mlRuns["dummy"] = main_ecg_history.current.mlRuns["dummy"][10000:20000:1]



## Feature Importance Functions

In [ ]:
import components.FeatureImportance as FI
import panel as pn
import pickle
pn.extension('plotly')
import os

def plot_feat_vis():
    model_directory = "./Models/monikit_model/"
    datapath = "./DataArtifacts/"
    comp = FI.FeatureImportance(model_directory=model_directory, traindatapath=datapath, testdatapath=datapath)
    # Load the feature importance from the saved file

    file_name = './DataArtifacts/impfeatures.json'
    
    if os.path.exists(file_name):
        print("FI file exists")
        comp.from_saved = True
        return pn.pane.Plotly(comp.plot_shap(file=file_name), scaling="auto")

    else:
        comp.calculate_shap()
        # Save the SHAP values to a file
        return pn.pane.Plotly(comp.plot_shap(file=file_name), scaling="auto")

row1 = pn.Row(plot_feat_vis(), height=800, sizing_mode='stretch_width')
feature_importance_content = pn.Column(row1, height=800,
                                       css_classes=["page-container", "main-content"])



# Processing panel

## Sidebar

### Input Selector

In [ ]:
# Input Type Selector # todo add default empty
input_type_select = pn.widgets.Select(
    name='Select Input Type', options=["",'Variable', 'File'], default=None)
# input_type_select


#### Variable Selector

In [ ]:
%%capture store_output
# Varable Selctor cells
# capture the store output to select from available ipy variables
# todo run this on selection of input type
%store


In [ ]:
# filter those to get a list 
try:
    var_options = [x[0] for x in store_output.stdout.split("\n")[1:-1]]
except:
    print("Jupyter unavailable")
    var_options = []
input_select = pn.widgets.Select(
    name='Select Input Variable', options=var_options)
# input_select


In [ ]:
# Single file selector
class SingleVariableFileSelectorParam(param.Parameterized):
    relPath = './DataArtifacts/*'
    # combine to total path
    path = os.path.join(absPath, relPath)    
    value = param.FileSelector(path=path, precedence=0.5, label="Select Filename")


file_processing_param = SingleVariableFileSelectorParam()
file_select = pn.Param(file_processing_param.param["value"], widgets={
                       'Select Filename': pn.widgets.Select}, show_name=True)


# file_select


#### Assembly

In [ ]:
import h5py

# Input loader button
input_loader = pn.widgets.Button(name='Import Data', button_type='primary')


def importDataProcessing(data,name,sfreq):
    print("importing data")
    # check if this data is a valid DataHistoryItem if so add it to the main history
    if isinstance(data, DataHistoryItem):
                print("Adding data to main history, recognised as DataHistoryItem")
                return main_ecg_history.addNew(data)
    # otherwise assume that its just raw data and add it to the main history
    else:
        return main_ecg_history.addNewFromDict(
            {"data": data, "name": name, "features": {"sfreq": sfreq}})

def loadInput(event):
    data = None
    name = ""
    sfreq = 0

    # try:
    if input_type_select.value == "Variable":
        %store -r $input_select.value
        data = eval(input_select.value)
        name = input_select.value
        
        
    elif input_type_select.value == "File":
        fileName = file_processing_param.value
        file = h5py.File(fileName, 'r')
        sfreq = file["header"]["sampling_rate"][0]
        name = fileName.split("/")[-1]
        prefix = name.split(".")[0]
        # Access the data
        data = file[prefix][:].flatten()
        file.close()   
        
        # data = np.load(fileName, allow_pickle=True)
        
    result = importDataProcessing(data,name,sfreq)
    # CHANGE CONTENT OF BUTTON TO REFLECT INPUT TYPE
    input_loader.name = "Input Loaded: " + \
        str(result.getID())
    # except Exception as e:
    #     input_loader.name = "Error Loading Input"
    #     print(e)


input_loader.on_click(loadInput)
input_loader


In [ ]:
# define input box
input_box = pn.Card(input_type_select, '',
                    input_loader, title="Quick Import",  sizing_mode='stretch_width')
input_box

#create a watcher that replaces the input box with a new one if the input type changes
@pn.depends(val=input_type_select.param.value, on_init=True, watch=False)
def updateInputAdditional(event):
    if input_type_select.value == "Variable":
        input_box[1] = input_select
    elif input_type_select.value == "File":
        input_box[1] = file_select
    else:
        input_box[1] = ""
input_type_select.param.watch(updateInputAdditional, 'value')

## Active Data Selector

In [ ]:

processing_active_data_selector_object = ECGHistorySelector(ecg_history=main_ecg_history, label="Select Active Data Object")
processing_active_data_selector = processing_active_data_selector_object.render()

def updateActiveDataValueProcessing(*events):
    print("updating active data value")
    processing_active_data_selector.value = main_ecg_history.current.getID()
    
def updateCurrentEcgHistoryProcessing(*events):
    print("updating current ecghistory")
    main_ecg_history.current = main_ecg_history.getHistoryFromID(processing_active_data_selector.value)


# update value if the main_ecg_data current changes
processing_active_data_selector_watcher = main_ecg_history.param.watch(
    updateActiveDataValueProcessing , "current", onlychanged=True)
# update the current data object in the main_ecg_history object if the selector changes
processing_active_data_selector.param.watch(
    updateCurrentEcgHistoryProcessing, "value")
# create a box for the selector
processing_active_data_selector_box = pn.Card(
    processing_active_data_selector, title="Active Data Selector",  sizing_mode='stretch_width')
processing_active_data_selector_box

### Process Selector

In [ ]:
# data settings for the Pipeline selection
pipelineBasePath = './Pipeline/*.py'
pipelineModuleBasePath = "Pipeline."
pipelineObj = None

# Single file selector


class SingleCommandFileSelectorParam(param.Parameterized):
    relPath = pipelineBasePath
    # combine to total path 
    path = os.path.join(absPath, relPath)    
    value = param.FileSelector(path=path, precedence=0.5)

    def getFunctionName(value):
        return value.split("/")[-1].split(".")[0] if value != None else None


action_param = SingleCommandFileSelectorParam
action_select = pn.Param(action_param.param['value'], widgets={
                         'Select Filename': pn.widgets.Select})

action_select


In [ ]:
# Pipeline arguemnts Select
# helper function to import the class part of the pipeline module
def importName(modulename, name):
    """ Import a named object from a module in the context of this function.
    """
    try:
        module = __import__(modulename, globals(), locals(), [name])
    except ImportError:
        print("Error importing module: " + modulename)
        return None
    return vars(module)[name]


def renderAdditionalPipelineArguments(*events):
    # get value from action_select
    pipeline = None
    if action_param.value is not None:
        pipeline = action_param.getFunctionName(action_param.value)

    if pipeline is not None:
        pipelineModulePath = pipelineModuleBasePath + pipeline
        pipeLineClass = importName(pipelineModulePath, pipeline)
        if pipeLineClass is None:
            print("Error importing pipeline class: " + pipeline)

        global pipelineObj
        pipelineObj = pipeLineClass()
        pipeLineInputs = pn.Param(pipelineObj.param)
        return pn.Card(pipeLineInputs, title="Pipeline Arguments")
    else:
        return pn.Card(title="Pipeline Arguments - Error")


# Install watcher for additional pipleline agruments on action select
#PipelineArgumentWatcher = action_param.param.watch(renderAdditionalPipelineArguments,"value", onlychanged=False)


In [ ]:
# main_ecg_history.addNewToExisting("ECG1", data = main_ecg_history.current.data, features={"peaks":[0,1]}, runs = None)

In [ ]:
runPipelineButton = pn.widgets.Button(
    name='Run Command', button_type='primary')


def runPipeline(event):
    try:
        if pipelineObj is not None:
            res = pipelineObj.run(main_ecg_history.current.data)
            # Now combine result with history
            print("A")
            id = main_ecg_history.current.getID()
            print("b")
            data = res["ecg"] if "ecg" in res else None
            print("c")
            features = res["features"] if "features" in res else None
            mlRuns = res["mlRuns"] if "mlRuns" in res else None
            print("d")
            print(features, mlRuns)
            main_ecg_history.addNewToExisting(id, data = data, features = features, runs = None)
            
            main_ecg_history.param.trigger('history')
            main_ecg_history.param.trigger('current')
            runPipelineButton.name = "Command Run!"
        else:
            print("No Pipeline Object")
            runPipelineButton.name = "Error Running Command"
    except:
        print("Error running command")
        runPipelineButton.name = "Error Running Command"


runPipelineButton.on_click(runPipeline)
action_box = pn.Card(action_select, renderAdditionalPipelineArguments,
                     runPipelineButton, title="Pipeline Selector", sizing_mode='stretch_width')


def updateAdditionalRenderings(*events):
    action_box[1] = renderAdditionalPipelineArguments()


action_param.param.watch(updateAdditionalRenderings, "value")
action_box


### Assembly

In [ ]:
# create a column that contains the input selector and the button
sidebar_processing = pn.Column(input_box, processing_active_data_selector_box, action_box, css_classes=["sidebar"])
sidebar_processing


## Main Content

### Data Definition


In [ ]:
# now the buttons to select the correct timescale data
time_scale_input_selector_1 = ECGHistorySelector(ecg_history=main_ecg_history, label='Select Input for Visualization 1').render()
time_scale_input_selector_2 = ECGHistorySelector(ecg_history=main_ecg_history, label='Select Input for Visualization 2').render()



In [ ]:
loadData()
# alter last entry so we can see difference #todo remove if dummy data is no longer needed
main_ecg_history.current.data = main_ecg_history.current.data[1000:10000:1]
# main_ecg_history.current.mlRuns["truth"] = main_ecg_history.current.mlRuns["truth"][1000:10000:1]
# main_ecg_history.current.mlRuns["dummy"] = main_ecg_history.current.mlRuns["dummy"][1000:10000:1]

processing_window_selector = pn.widgets.EditableRangeSlider(name='Processing Window', start=0, end=10000, value=(
    0, 10000), step=1, format="0[.]00", sizing_mode='stretch_width', css_classes=["window1"])


# TODO this should be dependent on the (first input)
@pn.depends(time_scale_input_selector_1, watch=True, on_init=True)
def updateProcessingWindowSelector(time_scale_selector=time_scale_input_selector_1):
    print("updating window selector view")
    # get current timescale data
    time_scale_obj = main_ecg_history.getHistoryFromID(time_scale_selector)
    time_scale_data = time_scale_obj.data

    # get count of datapoints
    data_count = len(time_scale_data)
    # get sampling rate 
    if "sfreq" in time_scale_obj.features:
        #rate = time_scale_obj.features["signalFreq"] change when visulisation respects frequency
        rate =1
    else:
        rate = 1
    # set start and end to 0 and count/sampling rate
    processing_window_selector.start = 0
    processing_window_selector.end = data_count/rate
    v1, v2 = processing_window_selector.value
    # TODO this should be dependent on the (first input)
    processing_window_selector.value = (max(v1, 0), min(v2, data_count/rate))


In [ ]:
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource, Scatter
from bokeh.models import CustomJS, TapTool
from bokeh.models import ColumnDataSource, Column
from bokeh.events import DoubleTap

def displayTimelinePlot(time_scale_selector, timeline_window=None, signalList=[], css_classes=[]):
   
    # step 1 get data from input selector
    data_stream = main_ecg_history.getHistoryFromID(time_scale_selector)

    # define timestamps based on data length and sampling rate
    freq = 1    
    if 'sfreq' in data_stream.features:
        freq = data_stream.features['sfreq']
    time_stamps = np.arange(0, int(len(data_stream.data)/1))
    # step 2 read the window selector
    if timeline_window is None:
        # then take the entire data stream
        timeline_window = [0, len(data_stream.data)]
    f_data_stream = data_stream.data
    f_time_stamps = time_stamps

    figureDataSourceDict = dict(
        time_stamps=f_time_stamps, ECG=f_data_stream.data)
    # check if peaks in signal
    figureScatterSourceDict = {}
    scatterSource = None #backup for when no peaks are found
    def updatePeakDataOnChange(attr, old, new):
        index = None
        # find index of changed peak
        #for i in range(len(new["peakLocations"])):
        #    if new[i] != old[i]:
        #        # update the peak
        #        index = i
        #        break
        #print("index of changed peak: ", index)
        #print("old: ", str(old))
        #print("new: ", str(new))
        #data_stream.updateFromDict(new)
        # set the new peak data
        #data_stream.features["peaks"] = 
        # trgger change
    if "peaks" in data_stream.features:
        # transform peaks so a 1 is at the peak location
        # filter peaks so they are unique
        figureScatterSourceDict["peakLocations"] = np.unique(data_stream.features["peaks"])
        figureScatterSourceDict["peaks"] = data_stream.data[figureScatterSourceDict["peakLocations"]]
        
        scatterSource = ColumnDataSource(data=figureScatterSourceDict)
        scatterSource.on_change('data', updatePeakDataOnChange)
        

    # define dictionary based on data
    cds = ColumnDataSource(data=figureDataSourceDict)
    cds.on_change('data', lambda attr, old, new: print("data changed"))
    # define tools 
    tools = 'tap,box_zoom,pan'
    x_range=(timeline_window[0], timeline_window[1])
    y_range =(min(f_data_stream.data), max(f_data_stream.data))
    #create custom tap tool with python callback
    testCallback = CustomJS(args=dict(lineData=cds, peakData=scatterSource, windowY=y_range, windowX=x_range), code=""" 
                            // check if peaks are present
                            console.log(windowY);
                            let event_data = cb_data;
                            console.log("event data", event_data);
                            let posX = event_data.geometries.x;
                            let posY = event_data.geometries.y;
                            console.log("tap at x: " + posX + " y: " + posY);
                            if (peakData == null){ return; }
                            // check if there is a close peak in peak data
                            console.log("peak data", peakData);
                            let peakX = Array.from(peakData.data.peakLocations);
                            let peakY = Array.from(peakData.data.peaks);
                            // calc how large the real glyph of the peak is, so we can check against it
                            let glyphSizeXpercent = 10/800; // 10/600 glyphsize/width
                            let glyphSizeYpercent = 10/600; // 10/600
                            let glyphSizeX = (windowX[1] - windowX[0]) * glyphSizeXpercent;
                            let glyphSizeY = (windowY[1] - windowY[0]) * glyphSizeYpercent;
                            
                            
                            // check if there is a peak close to the tap
                            let i = 0
                            let foundPeak = false;
                            for (i; i < peakX.length; i++){
                                // break if x is larger than posX
                                if (peakX[i] > posX+glyphSizeX/2){ break; }
                                // otherwise check if x is close to posX
                                if (Math.abs(peakX[i] - posX) < glyphSizeX/2){
                                    // check if y is close to posY
                                    if (Math.abs(peakY[i] - posY) < glyphSizeY/2){
                                        // then we have a peak close to the tap
                                        // and we can remove the peak
                                        console.log("removing peak at x: " + peakX[i] + " y: " + peakY[i]);
                                        console.log(peakX, peakY);
                                        peakX.splice(i, 1);
                                        peakY.splice(i, 1);  
                                        console.log(peakX, peakY);                                      
                                        foundPeak = true;
                                        break;
                                        
                                        }
                                        else{
                                        console.log(Math.abs(peakY[i] - posY), glyphSizeY/2);
                                        }
                                    }
                                    else{
                                    console.log(Math.abs(peakX[i] - posX), glyphSizeX/2);
                                }
                                    
                                }
                                
                                
                                        
                             
                            // otherwise we add a new peak at the point just after element i
                            if (!foundPeak){
                            peakX.splice(i, 0, posX);
                            peakY.splice(i, 0, posY);
                            }
                            
                                
                            
                            console.log('tap tool callback')                            
                            peakData.data = {peaks: peakY, peakLocations: peakX}
                            peakData.change.emit();                           
                            """)
    cTapTool = TapTool(callback=testCallback)
    
    p = figure(sizing_mode='stretch_width', title='', x_range=x_range, y_range=y_range, 
               css_classes=css_classes, tools= [cTapTool, 'box_zoom', 'pan'], plot_width=800, plot_height=300)

    p.line('time_stamps', 'ECG', source=cds)
    if len(figureScatterSourceDict) > 0:
        # register a callback vent for when a user clicks a scatter point
        updateScatterSelection = CustomJS(args=dict(source= scatterSource ), 
                                          code="""
                let inds = cb_obj.indices;
                let d1 = source.data; 
            
                d1['peaks'] = Array.from(d1['peaks']);
                let d2 = {}
                d2['peakLocations'] = []
                d2['peaks'] = []
                
                for (let i = 0; i < inds.length; i++) {
                    d2['peakLocations'].push(d1['peakLocations'][inds[i]])
                    d2['peaks'].push(d1['peaks'][inds[i]])
                }
                for (let elem of d2['peakLocations']) {
                    const index = d1['peakLocations'].indexOf(elem);
                    if (index > -1) { // only splice array when item is found
                    d1['peakLocations'].splice(index, 1); //2nd parameter means remove one item only
                    d1['peaks'].splice(index, 1); //2nd parameter means remove one item only
                    }
                }
                source.selected.indices = [] // reset selections
                source.change.emit();
                
            """)
                      
        
        scatterSource.selected.js_on_change('indices', updateScatterSelection)
        #scatterSource.selected.on_change('indices', testCallback)
        # scatterplot with circle indicators of peaks
        p.scatter('peakLocations', 'peaks', source= scatterSource , marker="circle", size=10, color="red")

    # render pane
    bk_pane = pn.pane.Bokeh(p, css_classes=css_classes,
                            sizing_mode='stretch_width')
    return bk_pane


@pn.depends(time_scale_input_selector_1, processing_window_selector)
def  buildTimelineVis1(time_scale_selector=time_scale_input_selector_1, timeline_window=processing_window_selector):
    
    try:
        res = displayTimelinePlot(time_scale_selector, timeline_window, css_classes=["vis1"])
        #res = None
        
    except Exception as e:
        print(e)
        res = None
    return res if res is not None else pn.pane.Markdown("No data available")


@pn.depends(time_scale_input_selector_2, processing_window_selector)
def buildTimelineVis2(time_scale_selector=time_scale_input_selector_2, timeline_window=processing_window_selector):
    res = displayTimelinePlot(time_scale_selector, timeline_window, css_classes=["vis2"])
    #res = None
    return res if res is not None else pn.pane.Markdown("No data available")


### Assembly

In [ ]:
main_processing = pn.Column(css_classes=["main-content"])
top_row_1 = pn.Row(time_scale_input_selector_1, height=60, sizing_mode='fixed')
top_row_2 = pn.Row(buildTimelineVis1, sizing_mode='stretch_width')
middle_row = pn.Row(processing_window_selector, sizing_mode='stretch_width')
bot_row_1 = pn.Row(time_scale_input_selector_2, sizing_mode='fixed', height=60)
bot_row_2 = pn.Row(buildTimelineVis2, sizing_mode='stretch_width')
main_processing.append(top_row_1)
main_processing.append(top_row_2)
main_processing.append(middle_row)
main_processing.append(bot_row_1)
main_processing.append(bot_row_2)


## Assembly

In [ ]:
processing_panel = pn.Row(
    sidebar_processing, main_processing, css_classes=["page-container"])
#processing_panel = pn.Row(sidebar_processing,css_classes = ["page-container"])


# Insights Panel

In [ ]:
from components.insights.shared import InsightsWindowSelector, InsightsTab, InsightsFileSelector


In [ ]:
from components.insights.timeDomainVis import InsightsTimeDomainTab, InsightsTimeDomainStats, InsightsTimeDomainVis


def render_insights_windowSelector():
    # a time scale data selector
    # TODO bind wether this is greyed out to value of inisigths file selector
    return pn.widgets.IntRangeSlider(name="Input Window", start=0, end=100, value=(0, 100), step=1)


def render_insights_frequency_domain_vis():
    return

def render_insights_features_30():
    return pn.Row(pn.pane.Plotly(loadfeature30data(), scaling="auto"))

def render_insights_features_60():
    return pn.Row(pn.pane.Plotly(loadfeature60data(), scaling="auto"))

def render_insights_features_120():
    return pn.Row(pn.pane.Plotly(loadfeature120data(), scaling="auto"))

def render_insights_frequency_domain_side():
    return


def render_insights_frequency_domain():
    return pn.Row(render_insights_frequency_domain_side, render_insights_frequency_domain_vis)


def render_insights_nonlinear_domain_vis():
    return


def render_insights_nonlinear_domain_side():
    return


def render_insights_nonlinear_domain():
    return pn.Row(render_insights_nonlinear_domain_side, render_insights_nonlinear_domain_vis)


def render_insights_timevar_domain_vis():
    return


def render_insights_timevar_domain_side():
    return


def render_insights_timevar_domain():
    return pn.Row(render_insights_timevar_domain_side, render_insights_timevar_domain_vis)


def render_insights_panel_elem():
    # Row 1
    fileSelector = InsightsFileSelector(main_ecg_history)
    windowSelector = InsightsWindowSelector(fileSelector, main_ecg_history)

    # timeDomainButton = pn.widgets.Button(
    #     name="Time-Domain", button_type="primary", width=120)
    feature30Button = pn.widgets.Button(
        name="30s Features", button_type="primary", width=120)
    feature60Button = pn.widgets.Button(
        name="60s Features", button_type="primary", width=120)
    feature120Button = pn.widgets.Button(
        name="120s Features", button_type="primary", width=120)
    timeDom = InsightsTimeDomainTab(fileSelectorRef=fileSelector,
                                    windowSelectorRef=windowSelector,
                                    data_history=main_ecg_history)
    # freqDomainButton = pn.widgets.Button(
    #     name="Frequency-Domain", button_type="default", width=120)
    # nonlinDomainButton = pn.widgets.Button(
    #     name="Nonlinear-Domain", button_type="default", width=120)
    # timeVarDomainButton = pn.widgets.Button(
    #     name="Time-Varying-Domain", button_type="default", width=120)
    # buttonList = [timeDomainButton, freqDomainButton,
    #               nonlinDomainButton, timeVarDomainButton]
    buttonList = [feature30Button, feature60Button,
                  feature120Button]
    
    selectorList = [fileSelector.getRender, windowSelector.getRender]

    def restoreButtonStates():
        for button in buttonList:
            button.button_type = "default"

    
    top_row = pn.FlexBox(pn.Row(*buttonList), css_classes=["top-row"], justify_content="space-between", flex_wrap="nowrap")
    main_row = pn.Row(render_insights_features_30)
    # register switches on button press

    # def switchToTimeDomain(event):
    #     restoreButtonStates()
    #     timeDomainButton.button_type = "primary"
    #     main_row[0].loading = True
    #     main_row[0] = timeDom.getRender
    #     main_row[0].loading = False

    # def switchToFreqDomain(event):
    #     restoreButtonStates()
    #     freqDomainButton.button_type = "primary"
    #     main_row[0].loading = True
    #     main_row[0] = render_insights_frequency_domain
    #     main_row[0].loading = False

    # def switchToNonLinDomain(event):
    #     restoreButtonStates()
    #     nonlinDomainButton.button_type = "primary"
    #     main_row[0].loading = True
    #     main_row[0] = render_insights_nonlinear_domain
    #     main_row[0].loading = False

    # def switchToTimeVarDomain(event):
    #     restoreButtonStates()
    #     timeVarDomainButton.button_type = "primary"
    #     main_row[0].loading = True
    #     main_row[0] = render_insights_timevar_domain
    #     main_row[0].loading = False

    def switchToFeature30(event):
        restoreButtonStates()
        feature30Button.button_type = "primary"
        main_row[0].loading = True
        main_row[0] = render_insights_features_30
        main_row[0].loading = False

    def switchToFeature60(event):
        restoreButtonStates()
        feature60Button.button_type = "primary"
        main_row[0].loading = True
        main_row[0] = render_insights_features_60
        main_row[0].loading = False

    def switchToFeature120(event):
        restoreButtonStates()
        feature120Button.button_type = "primary"
        main_row[0].loading = True
        main_row[0] = render_insights_features_120
        main_row[0].loading = False

    # register clicks
    # timeDomainButton.on_click(switchToTimeDomain)
    # freqDomainButton.on_click(switchToFreqDomain)
    # nonlinDomainButton.on_click(switchToNonLinDomain)
    # timeVarDomainButton.on_click(switchToTimeVarDomain)

    feature30Button.on_click(switchToFeature30)
    feature60Button.on_click(switchToFeature60)
    feature120Button.on_click(switchToFeature120)

    # render the window selector param, so it has a value_throttled property
    res = pn.Column(top_row, main_row)
    # file selector
    # domain Selector
    # Row 2
    # Stats
    # Visualisation
    return res


### Assembly

In [ ]:
insights_panel = pn.Column(render_insights_panel_elem(
), css_classes=["page-container"])
#insights_panel

feature_importance_panel = pn.Column(feature_importance_content, css_classes=["page-container"])
#feature_importance_panel

# ML Panel

## Top Nav

### Input Selector

In [ ]:
# now the buttons to select the correct timescale data
time_scale_input_selector_ml_object = ECGHistorySelector(main_ecg_history, label = "Select Input")
time_scale_input_selector_ml = time_scale_input_selector_ml_object.render()



In [ ]:
# Single transformation File Selector
class SingleTransformationScriptSelectorParam(param.Parameterized):
    relPath = './MLTransformation/*'
    # combine to total path 
    path = os.path.join(absPath, relPath)    
    value = param.FileSelector(path=path, precedence=0.5, label="Select Transformation Script")



ml_transformation_param = SingleTransformationScriptSelectorParam()
ml_transformation_select = pn.Param(ml_transformation_param.param["value"], widgets={
                       'Select Filename': pn.widgets.Select}, show_name=False)


def getFunctionName(value):
    return value.split("/")[-1].split(".")[0] if value != None else None




In [ ]:
getFunctionName(ml_transformation_param.value)

### Model Selector + Run

In [2]:
# model eval type -> TODO this should be a dropdown
import torch
model_eval_type = "torch"


class SingleModelFileSelectorParam(param.Parameterized):
    relPath = './Models/*'
    # combine to total path 
    path = os.path.join(absPath, relPath)        
    value = param.FileSelector(path=path, precedence=0.5, default=None)
    


file_param = SingleModelFileSelectorParam()
file_select_ml = pn.Param(file_param.param["value"], widgets={
                          'Select Filename': pn.widgets.Select}, show_name=False)

buttonEvalML = pn.widgets.Button(
    name='Evaluate with Model', button_type='primary')

transformationBasePath = './MLTransformation/*.py'
transformationModuleBasePath = "MLTransformation."
transformationObj = None

def evaluateModel(event):
    # try:
        # get data from input selector
    print("evaluateModelNew")
    history_id = time_scale_input_selector_ml.value
    print("history_id: " + history_id)
    if history_id == None:
        print("Error: history_id is None")
        return
    data = main_ecg_history.getHistoryFromID(
        history_id)   
    print("data: ")   
    data = data.data
    print("data_len: " + str(len(data)))
    print(getFunctionName(ml_transformation_param.value))
    prediction = None
    
    print(ml_transformation_param.value)
    transformationName = None
    if ml_transformation_param.value!=None:
        transformationName = getFunctionName(ml_transformation_param.value)
    if transformationName!=None:
        
        print("transformationName: " + transformationName)
        transformationModulePath = transformationModuleBasePath + transformationName
        print("transformationModulePath: " + transformationModulePath)
        transformationClass = importName(transformationModulePath, transformationName)
        if  transformationClass is None:            
            print("Error importing  transformation class: " +  transformationName)

        global transformationObj
        transformationObj = transformationClass()
    
        prediction = transformationObj.run()
        prediction_dict = {transformationName +"_"+str(getFunctionName(file_param.value)): prediction}
        print("prediction_dict: " + str(prediction_dict))
        # add prediction to history
        res = main_ecg_history.addNewToExisting(history_id, data= None, features=None,runs=prediction_dict )
        print(res.mlRuns.keys())
        main_ecg_history.param.trigger('history')
        main_ecg_history.param.trigger('current')
        print("HERE")
        buttonEvalML.name = "Model Evaluation: " + str(res.getID())
    # except:
    #     buttonEvalML.name = "Error Running Model Evaluation"
    #     print("Error running model evaluation")
    

buttonEvalML.on_click(evaluateModel)


SyntaxError: invalid syntax (2479803525.py, line 59)

In [1]:
# import MLTransformation.monikit as mml
# eval = mml.monikit()
# preds = eval.run(model_file="/Users/shivam/Documents/GitHub/HRVInvestigator/Models/monikit_model/")
# print(preds)

starting to evaluate model


ModuleNotFoundError: No module named 'keras_self_attention'

## Main Content

In [ ]:
eval_window_selector = pn.widgets.EditableRangeSlider(name='Evaluation Window', start=0, end=100, value=(
    0, 100), step=1, format="0[.]00", sizing_mode='stretch_width')

eval_file_selector_1 = ECGHistorySelector(main_ecg_history, label = "Select Evaluation File").render()
eval_run_selector_1 = ECGRunSelector(main_ecg_history, label = "Select Evaluation Run", hook = eval_file_selector_1).render()
eval_file_selector_2= ECGHistorySelector(main_ecg_history, label = "Select Evaluation File").render()
eval_run_selector_2 = ECGRunSelector(main_ecg_history, label = "Select Evaluation Run", hook = eval_file_selector_2).render()


@pn.depends(eval_file_selector_1, watch=True)
def updateProcessingWindowSelectorML(time_scale_selector=time_scale_input_selector_ml):
    print("updating window selector view")
    # get current timescale data
    time_scale_data = main_ecg_history.getHistoryFromID(
        time_scale_selector).data

    # get count of datapoints
    data_count = len(time_scale_data)
    # get sampling rate #TODO 
    rate = 1
    # set start and end to 0 and count/sampling rate
    eval_window_selector.start = 0
    eval_window_selector.end = data_count/rate
    v1, v2 = eval_window_selector.value
    
    eval_window_selector.value = (max(v1, 0), min(v2, data_count/rate))


### Model 1


#### Timeline Vis

In [ ]:

def buildEvalTimelineVis(time_scale_selector=time_scale_input_selector_ml, run=eval_run_selector_1, timeline_window=eval_window_selector):
    # return a bokeh Pane of a plot using both the data and the run information
    
    #get data
    data_stream = main_ecg_history.getHistoryFromID(time_scale_selector)
    
    #define frequency and timestamps
    req = 1    
    if 'sFreq' in data_stream.features:
        freq = data_stream.features['sFreq']
    time_stamps = np.arange(0, int(len(data_stream.data)/1))
    
    #access timeline window
    timeline= [0,1]
    if timeline_window is None:
        # then take the entire data stream
        timeline = [0, len(data_stream.data)]
    else:
        timeline = timeline_window
        
    # create og data source dict
    figureDataSourceDict = dict(
        time_stamps=time_stamps, ECG=data_stream.data)
    
    # check if truth is in mlRuns
    if 'truth' in data_stream.mlRuns:
        figureDataSourceDict['truth'] = data_stream.mlRuns['truth']
    if run is not None:
        # check if run is in mlRuns
        if run in data_stream.mlRuns:
            figureDataSourceDict["run"] = data_stream.mlRuns[run]
            
    # define dictionary based on data
    cds = ColumnDataSource(data=figureDataSourceDict)
    # define tools 
    tools = 'tap,box_zoom,pan'
    # define figure
    p = figure(plot_width=800, plot_height=400, tools=tools, title="ML Evaluation", x_axis_label='Time (s)', x_range=(timeline[0], timeline[1]))
    # add line for ECG
    p.line(x='time_stamps', y='ECG', source=cds, line_width=2, line_alpha=0.8, color='blue')
    # add scatter for truth
    p.scatter(x='time_stamps', y='truth', source=cds, color='green')
    # add scatter for run
    p.scatter(x='time_stamps', y='run', source=cds, color='red')
    
    bk_pane = pn.pane.Bokeh(p,
                            sizing_mode='stretch_width')
    return bk_pane
    


#### Stats

In [ ]:
from sklearn.metrics import confusion_matrix
import pandas as pd
def mlEvalVisStats(sel_file=eval_file_selector_1, sel_run=eval_run_selector_1, sel_window=eval_window_selector):
    try:
        # check if file exists
        
        if sel_file is None  or sel_run is None or sel_window is None:
            print(sel_file is None, sel_run is None, sel_window is None)
            return pn.pane.Markdown("Stats unavailable,Please select a file, run and window", style={'font-family': "serif"})
        
        data_obj = main_ecg_history.getHistoryFromID(sel_file)
        
        if (sel_run not in data_obj.mlRuns or "truth" not in data_obj.mlRuns.keys()):
            return pn.pane.Markdown("Ground Truth or Prediction Unavailable", style={'font-family': "serif"})
        
        start = max(0, sel_window[0])
        end = min(len(data_obj.data), sel_window[1])
        y_true = data_obj.mlRuns["truth"][start:end]
        y_pred = data_obj.mlRuns[sel_run][start:end]
        
        conf = pd.DataFrame(confusion_matrix(y_true, y_pred))
        # calculate accurcy from confusion matrix 
        accuracy = (conf.iloc[0,0] + conf.iloc[1,1]) / conf.sum().sum()

        confusion = conf
        confusion_matrix_rep = pn.widgets.DataFrame(
            confusion, autosize_mode='fit_columns', width=300)
        accuracy_rep = pn.pane.Markdown(
            'Accuracy:' + str(accuracy), style={'font-family': "serif"})
        return pn.Card(accuracy_rep, confusion_matrix_rep, title="Stats")   
    except:
        return pn.pane.Markdown("Error while rendering Stats", style={'font-family': "serif"})


@pn.depends(eval_file_selector_1,eval_run_selector_1, eval_window_selector)
def buildEvalVis1(file_selector=time_scale_input_selector_1, run_selector = eval_run_selector_1, timeline_window=eval_window_selector):
    
    try:
        res = pn.Row(mlEvalVisStats(file_selector,run_selector, timeline_window), 
                     buildEvalTimelineVis(file_selector, run_selector, timeline_window))
        return res
    except Exception as e:
        print(e)
        res = pn.pane.Markdown("No Visualization available. Error while rendering")
    return res if res is not None else pn.pane.Markdown("No data available")


@pn.depends(eval_file_selector_2,eval_run_selector_2, eval_window_selector)
def buildEvalVis2(file_selector=time_scale_input_selector_1, run_selector = eval_run_selector_1, timeline_window=eval_window_selector):
    try:
        res = pn.Row(mlEvalVisStats(file_selector,run_selector, timeline_window), 
                     buildEvalTimelineVis(file_selector, run_selector, timeline_window))
        return res
    except Exception as e:
        print(e)
        res = pn.pane.Markdown("No Visualization available. Error while rendering")
    return res if res is not None else pn.pane.Markdown("No data available")





### Model 2

### Assembly

In [ ]:
ml_row1 = pn.Row(time_scale_input_selector_ml,ml_transformation_select, file_select_ml,
                 buttonEvalML, sizing_mode='stretch_width')
ml_row1_5 = pn.Row(eval_file_selector_1,eval_run_selector_1, sizing_mode='stretch_width')
ml_row2 = buildEvalVis1
# ml_row3 = pn.Row(eval_window_selector, sizing_mode='stretch_width')
# ml_row3_5 = pn.Row(eval_file_selector_2,eval_run_selector_2, sizing_mode='stretch_width')
# ml_row4 = buildEvalVis2

ml_panel = pn.Column(ml_row1,ml_row1_5, ml_row2, 
                     css_classes=["page-container", "main-content"])


# Data Panel

In [ ]:
# Data Selector
# TODO need dummy model to test this and perhaps remove the time series restriction
data_selector = pn.widgets.Select(name='Select Active Input', options=[x.getID(
) for x in main_ecg_history.history], value=main_ecg_history.current.getID())

# Data Selector Watcher


def updateOption(*events):
    data_selector.options = [x.getID() for x in main_ecg_history.history]
# Data Active Watcher


def updateActiveData(*events):
    data_selector.value = main_ecg_history.current.getID()


def updateActiveSelectedData(*events):
    print("updating active data")
    main_ecg_history.current = main_ecg_history.getHistoryFromID(
        data_selector.value)


dataUpdateWatcher = data_selector.param.watch(
    updateActiveSelectedData, "value", onlychanged=False)
dataOptionUpdateWatcher = main_ecg_history.param.watch(
    updateOption, "history", onlychanged=False)
dataOptionUpdateWatcher = main_ecg_history.param.watch(
    updateActiveData, "history", onlychanged=False)

data_selector


In [ ]:
# Data Inspector and rename
# TODO get back here when data Structure is redone
@pn.depends(data_selector)
def showCurrentData(*events):
    print("selected data has changed")
    return pn.Param(main_ecg_history.current.param, widgets={
        'data': {'widget_type': pn.widgets.ArrayInput, 'disabled': True},
        'name': {'widget_type': pn.widgets.TextInput},


    }, parameters=['data', 'name', "features"], name="Data Inspector", sizing_mode='stretch_width', expand=True)


In [ ]:
# Data Saver and Importer
dataPath = "./DataArtifacts/"
saveDataButton = pn.widgets.Button(name='Save Data', button_type='primary')
loadDataButton = pn.widgets.Button(name='Load Data', button_type='primary')
loadDataInput = pn.widgets.FileInput(accept='.pkl', multiple=False)
saveData = None


def saveData(*events):
    data_obj = main_ecg_history.current
    name = data_obj.getID()
    # pickle the data
    with open(dataPath + name + '.pkl', 'wb') as f:
        pickle.dump(main_ecg_history.current.getSaveDict(), f)

    global savedData 
    savedData= main_ecg_history.current.getSaveDict()
    %store savedData
    saveDataButton.name = "Data Saved"


def loadData(*events):
    try:
        # get the file name
        filename = loadDataInput.filename    
        
        if filename is None:
            loadDataButton.name = "Error: Set Filename"
        
        # load the data
        with open(dataPath + filename, 'rb') as f:
            data = pickle.load(f)
            main_ecg_history.addNewFromDict(data)
        # add the data to the history

        loadDataButton.name = "Data Loaded"
    except:
        loadDataButton.name = "Error: make sure to load from DataArtifacts folder"
        


saveDataButton.on_click(saveData)
loadDataButton.on_click(loadData)

dataLoaderArea = pn.Row(saveDataButton, loadDataButton, loadDataInput)
dataLoaderArea


## Assembly

In [ ]:
from panel.interact import interact
data_panel = pn.Column(data_selector, showCurrentData, dataLoaderArea, css_classes=[
                       "page-container", "main-content"])


def updateMainHistory(*events):
    """This function is used to update the main history when the data inside is changed, by emitting a trigger event on the history, when an element is changed
    """
    main_ecg_history.history = main_ecg_history.history
    main_ecg_history.param.trigger("history")
    print("updating main history" + str(main_ecg_history.current.getID()))


main_ecg_history.current.param.watch(
    updateMainHistory, ["name", "features", 'data'], onlychanged=False)
data_panel


# Console Panel 

In [ ]:
console_terminal = pn.widgets.Terminal(
    options={"cursorBlink": True}, height=500, width=500, sizing_mode='stretch_width')
sys.stdout = console_terminal
print("Welcome to the ECG Console")
# console_terminal


## Assembly


In [ ]:
console_panel = pn.Column(console_terminal, css_classes=[
                          "page-container", "main-content"])


# Orchestration

## Assembly

In [ ]:

# instead of tabs use buttons that change the layout
# create a top bar with a button for each panel with data and console beeing rightbound
process_button = pn.widgets.Button(
    name='Process', button_type='primary', width=150)
insights_button = pn.widgets.Button(
    name='Insights', button_type='light', width=150)
machine_learning_button = pn.widgets.Button(
    name='Machine Learning', button_type='light', width=150)
feature_importance_button = pn.widgets.Button(
    name='Feature Importance', button_type='light', width=150)
data_button = pn.widgets.Button(
    name='Data', button_type='light', align='end', width=150)
console_button = pn.widgets.Button(
    name='Console', button_type='light', align='end', width=150)
buttonList = [process_button, insights_button,
              machine_learning_button,feature_importance_button, data_button, console_button]


def resetAllButtons():
    for button in buttonList:
        button.button_type = "default"


top_bar = pn.FlexBox(pn.Row(process_button, insights_button, machine_learning_button, feature_importance_button), pn.Row(
    data_button, console_button), justify_content='space-between', sizing_mode='stretch_width', css_classes=["top-bar"])



layout = pn.Column(top_bar, processing_panel, sizing_mode='stretch_width')
# on click of buttons change the layout


def openProcessing(*events):
    resetAllButtons()
    process_button.button_type = 'primary'
    layout[1].loading = True
    layout[1] = processing_panel
    layout[1].loading = False


def openFeatureImportance(*events):
    resetAllButtons()
    feature_importance_button.button_type = 'primary'
    layout[1].loading = True
    layout[1] = feature_importance_panel
    layout[1].loading = False


def openML(*events):
    resetAllButtons()
    machine_learning_button.button_type = 'primary'
    layout[1].loading = True
    layout[1] = ml_panel
    layout[1].loading = False

def openInsights(*events):
    resetAllButtons()
    insights_button.button_type = 'primary'
    layout[1].loading = True
    layout[1] = insights_panel
    layout[1].loading = False


def openData(*events):
    resetAllButtons()
    data_button.button_type = 'primary'
    layout[1].loading = True
    layout[1] = data_panel
    layout[1].loading = False


def openConsole(*events):
    resetAllButtons()
    console_button.button_type = 'primary'
    layout[1].loading = True
    layout[1] = console_panel
    layout[1].loading = False


process_button.on_click(openProcessing)
insights_button.on_click(openInsights)
machine_learning_button.on_click(openML)
data_button.on_click(openData)
console_button.on_click(openConsole)
feature_importance_button.on_click(openFeatureImportance)

layout.servable()
# we can serve via cmd using : panel serve test.ipynb
